## Setup and Import Libraries

In [1]:
import os
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from dotenv import load_dotenv

import warnings
warnings.filterwarnings("ignore")

In [2]:
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [4]:
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
llm = init_chat_model(model="groq:gemma2-9b-it")

## Load and Chunk the Document

In [5]:
loader = TextLoader(file_path="langchain_rag_dataset.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(documents=documents)

chunks

[Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain is an open-source framework designed to simplify the development of applications using large language models (LLMs).\nLangChain provides abstractions for working with prompts, chains, memory, and agents, making it easier to build complex LLM-based systems.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='The framework supports integration with various vector databases like FAISS and Chroma for semantic retrieval.\nLangChain enables Retrieval-Augmented Generation (RAG) by allowing developers to fetch relevant context before generating responses.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
 Document(metadata={'

## Create Vector Store and MMR Retriever

In [6]:
vector_store = FAISS.from_documents(
    documents=chunks, 
    embedding=embeddings
)

retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3}
)

retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002B822359A00>, search_type='mmr', search_kwargs={'k': 3})

## Prompt and Rag Chain

In [7]:
prompt = PromptTemplate.from_template("""
Answer the question based on the context provided.

Context:
{context}

Question: {input}
""")

prompt

PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the question based on the context provided.\n\nContext:\n{context}\n\nQuestion: {input}\n')

In [8]:
document_chain = create_stuff_documents_chain(
    llm=llm, 
    prompt=prompt
)

rag_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=document_chain
)

## Query

In [9]:
query = "How does LangChain support agents and memory?"

response = rag_chain.invoke(input={"input": query})

print("✅ Answer:\n", response["answer"])

✅ Answer:
 LangChain supports agents in several ways:

* **Tool integration:** Agents can utilize tools like calculators, search APIs, or custom functions based on their instructions.
* **External API and database interaction:** Agents can connect with external APIs and databases, expanding their capabilities beyond text-based processing.
* **Tool selection and execution:** LangChain allows LLMs to act as agents, deciding which tool to use and in what sequence to complete a task.

For memory, LangChain offers:

* **Conversational memory:**  `ConversationBufferMemory`  retains previous interactions for more coherent multi-turn conversations.
* **Summarization memory:** `ConversationSummaryMemory`  provides a summarized view of past interactions, useful for longer conversations. 


Together, these features enable LangChain to build sophisticated, context-aware agents capable of handling complex tasks. 



In [10]:
response

{'input': 'How does LangChain support agents and memory?',
 'context': [Document(id='e9d7ca86-ef61-4d5d-900c-39175d548e19', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
  Document(id='f5b54435-89aa-4d40-a19e-d6a445daf3a9', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain agents can interact with external APIs and databases, enhancing the capabilities of LLM-powered applications.\nRAG pipelines in LangChain involve document loading, splitting, embedding, retrieval, and LLM-based response generation.'),
  Document(id='45f027da-bf57-4973-a4e1-3326fb364caf', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain allows LLMs to act as agents that decide which tool to call and in what order duri